### environment setup

In [ ]:
# install necessory packages
!conda install -c anaconda tk -y
!conda install -c anaconda pillow -y
!conda install -c conda-forge hdf5storage -y

### tkinter 

#### load images for all towns

In [ ]:
import hdf5storage

save_dir = "/media/statespace/Spatial/sptialworkspace/spatialfMRI/simulation/carla_ws/label_road/"
matdic = hdf5storage.loadmat(save_dir+"rgb_images_array.mat")
rgb_images_array = matdic["rgb_images_array"]
print('(num_towns, num_images, C, H, W)', rgb_images_array.shape)

# i_town is from 0 to 7 (all 8 towns)
i_town = 0
rgb_images = rgb_images_array[i_town]

#### save images to seperated files for each town


In [ ]:
import hdf5storage
import numpy as np

for i_town in np.arange(0, 8, 1, dtype=int):
  i_file_name = "rgb_images_array_town{i_town:02}".format(i_town=i_town+1)
  matdic = {i_file_name:rgb_images_array[i_town]}
  hdf5storage.savemat(save_dir+i_file_name+".mat", matdic)

#### load images for each town


In [1]:
import hdf5storage

# i_town is from 0 to 7 (all 8 towns)
i_town = 0

save_dir = "/media/statespace/Spatial/sptialworkspace/spatialfMRI/simulation/carla_ws/label_road/"
i_file_name = "rgb_images_array_town{i_town:02}".format(i_town=i_town+1)
print("load file:", i_file_name)

matdic = hdf5storage.loadmat(save_dir+i_file_name+".mat")
rgb_images = matdic[i_file_name]
print('(num_images, C, H, W)', rgb_images.shape)

load file: rgb_images_array_town01
(num_images, C, H, W) (200, 3, 900, 1600)


#### label gui launch

In [2]:
# importing required packages
import tkinter
from PIL import ImageTk, Image
import os
import numpy as np
import sys

# creating main window
window = tkinter.Tk()

# place windown on the center
screen_width = window.winfo_screenwidth()  # Width of the screen
screen_height = window.winfo_screenheight() # Height of the screen

width = screen_width*0.6 # Width 
height = screen_height*0.8 # Height

x = (screen_width/2) - (width/2)
y = (screen_height/2) - (height/2)

window.geometry('%dx%d+%d+%d' % (width, height, x, y))
window.title('Labeling Road Structure')

# opens the image
global i_rgb_image
i_rgb_image = 0
# imagetk read first image
temp_rgb_image = rgb_images[i_rgb_image] #CxHxW format
temp_rgb_image = np.transpose(temp_rgb_image, (1, 2, 0)) #HxWxC
pil_temp_rgb_image = Image.fromarray((temp_rgb_image).astype(np.uint8))

# imagetk
img_town_temp = pil_temp_rgb_image.resize((int(screen_width*0.6), int(screen_height*0.5)), Image.Resampling.LANCZOS)
img_town_cam = ImageTk.PhotoImage(img_town_temp)
# i_rgb_image = i_rgb_image + 1

# read any image
# img_town_cam = Image.open("rgb_town07.jpg")
# img_town_cam = img_town_cam.resize((int(screen_width*0.6), int(screen_height*0.5)), Image.Resampling.LANCZOS)
# img_town_cam = ImageTk.PhotoImage(img_town_cam)

# create canvas
canvas_town_cam = tkinter.Canvas(window, width=int(screen_width*0.6), height=int(screen_height*0.5))
img_town_container = canvas_town_cam.create_image(0,0, anchor='nw', image=img_town_cam)
canvas_town_cam.pack(side='top')

# opens the image
img_road_label = Image.open("roadtypes.png")
img_road_label = img_road_label.resize((int(screen_width*0.6), int(screen_height*0.18)), Image.Resampling.LANCZOS)
img_road_label = ImageTk.PhotoImage(img_road_label)

# create canvas
canvas_road_label = tkinter.Canvas(window, width=int(screen_width*0.6), height=int(screen_height*0.2))
img_road_label_container = canvas_road_label.create_image(0, 0, anchor='nw', image=img_road_label)
canvas_road_label.pack(side='top')

# # user information
# tkinter.Label(window, text='Types:', font=('Helvetica bold', 40)).place(x=0, y=750)
rb_rely = 0.90
rb_height = int(screen_height*0.0024)
v = tkinter.StringVar(window, "1")
variable_radio = tkinter.IntVar()

tkinter.Radiobutton(window, text = "Straight", variable = variable_radio, font=('Helvetica bold', 16),
            value = "1", indicator = 0, height=rb_height, width=int(screen_width*0.008),
            background = "white").place(relx = 0.09, rely = rb_rely, anchor = tkinter.CENTER)

tkinter.Radiobutton(window, text = "Left or right turn", variable = variable_radio, font=('Helvetica bold', 16),
            value = "2", indicator = 0, height=rb_height, width=int(screen_width*0.015),
            background = "white").place(relx = 0.31, rely = rb_rely, anchor = tkinter.CENTER)

tkinter.Radiobutton(window, text = "T junction", variable = variable_radio, font=('Helvetica bold', 16),
            value = "3", indicator = 0, height=rb_height, width=int(screen_width*0.013),
            background = "white").place(relx = 0.63, rely = rb_rely, anchor = tkinter.CENTER)

tkinter.Radiobutton(window, text = "Crossroads", variable = variable_radio, font=('Helvetica bold', 16),
            value = "4", indicator = 0, height=rb_height, width=int(screen_width*0.010),
            background = "white").place(relx = 0.9, rely = rb_rely, anchor = tkinter.CENTER)

# tkinter.Radiobutton(window, text = "Others", variable = variable_radio, font=('Helvetica bold', 16),
#             value = "5", indicator = 0, height=rb_height, width=int(screen_width*0.010),
#             background = "white").place(relx = 0.89, rely = rb_rely, anchor = tkinter.CENTER)

# scale
variable_scale = tkinter.DoubleVar()
s2 = tkinter.Scale(window, variable = variable_scale,
           from_ = 1, to = 10, length=int(screen_width*0.15),
           orient = tkinter.HORIZONTAL).place(relx = 0.3, rely = 0.96, anchor = tkinter.CENTER)

lalel_scale = tkinter.Label(window, text = "Degree of belief:").place(relx = 0.1, rely = 0.97, anchor = tkinter.CENTER)

road_structure_list = ['straight', 'Left or right turn', 'T junction', 'Crossroads', 'Others']

road_labels_array = np.zeros((len(road_structure_list), rgb_images.shape[0]))

# process func
def labeling_road_func():

    global i_rgb_image

    print("--------------------------")
    print("image index:", i_rgb_image)
    # print("radio variable:", variable_radio.get())
    print("road:", road_structure_list[variable_radio.get()-1])
    print("belief:", int(variable_scale.get()))

    road_labels_array[variable_radio.get()-1][i_rgb_image] = int(variable_scale.get())

    # for the next image index
    i_rgb_image = i_rgb_image + 1

    if i_rgb_image == rgb_images.shape[0]:
        print('Finish town-0'+str(i_town+1)+' labeling.')
        window.destroy()
        sys.exit()

    # read new image
    temp_rgb_image = rgb_images[i_rgb_image] #CxHxW format
    temp_rgb_image = np.transpose(temp_rgb_image, (1, 2, 0)) #HxWxC
    pil_temp_rgb_image = Image.fromarray((temp_rgb_image).astype(np.uint8))

    # imagetk
    img_town_temp = pil_temp_rgb_image.resize((int(screen_width*0.6), int(screen_height*0.5)), Image.Resampling.LANCZOS)
    img_town_temp = ImageTk.PhotoImage(img_town_temp)
    
    # update
    canvas_town_cam.itemconfig(img_town_container,image=img_town_temp)
    canvas_town_cam.imgref = img_town_temp


# next image
btn = tkinter.Button(window, text = 'Next image', command = labeling_road_func).place(relx = 0.5, rely = 0.97, anchor = tkinter.CENTER)

# running the application
window.mainloop()


#### save

In [ ]:
from scipy.io import savemat

matdic = {"road_labels_array":road_labels_array}
save_dir = "/media/statespace/Spatial/sptialworkspace/spatialfMRI/simulation/carla_ws/label_road/"
savemat(save_dir+"road_labels_town_"+str(i_town)+".mat", matdic)
